# Divide and conquer
### method of computing the convex hull of a finite set of points in 2-dimensional space


In [ ]:
from ipynb.fs.full.drawing_tool import *
%matplotlib notebook
Tolerance = 10e-12

## 1. Auxiliary functions:

In [ ]:
def orient(a, b, c):
    det_ = a[0] * b[1] + b[0] * c[1] + c[0] * a[1] - c[0] * b[1] - b[0] * a[1] - a[0] * c[1]
    if det_ > Tolerance:
        return 1
    elif det_ < -Tolerance:
        return -1
    else:
        return 0

In [ ]:
def find_max_x(tab):
    max_x = -float('inf')
    max_ind = -1
    for x in range(len(tab)):
        if max_x < tab[x][0]:
            max_x = tab[x][0]
            max_ind = x
    return max_ind

In [ ]:
def find_min_x(tab):
    min_x = float('inf')
    min_ind = -1
    for x in range(len(tab)):
        if min_x > tab[x][0]:
            min_x = tab[x][0]
            min_ind = x
    return min_ind

In [ ]:
def points_connect_down(a, b, hull1, hull2):
    n1 = len(hull1)
    n2 = len(hull2)
    flag1, flag2 = False, False
    while not flag1 or not flag2:
        flag1, flag2 = True, True
        if n1 != 1:
            while orient(hull1[(a - 1) % n1], hull1[a % n1], hull2[b % n2]) <= 0:
                a -= 1
                flag1 = False
        if n2 != 1:
            while orient(hull1[a % n1], hull2[b % n2], hull2[(b + 1) % n2]) <= 0:
                b += 1
                flag2 = False
    return a % n1, b % n2

In [ ]:
def points_connect_up(a, b, hull1, hull2):
    n1 = len(hull1)
    n2 = len(hull2)
    flag1, flag2 = False, False
    while not flag1 or not flag2:
        flag1, flag2 = True, True
        if n1 != 1:
            while orient(hull1[a % n1], hull1[(a + 1) % n1], hull2[b % n2]) <= 0:
                a += 1
                flag1 = False
        if n2 != 1:
            while orient(hull1[a % n1], hull2[(b - 1) % n2], hull2[b % n2]) <= 0:
                b -= 1
                flag2 = False
    return a % n1, b % n2

In [ ]:
def trivial(points, first, last):
# determining the convex hull for <= 3 points
    if last - first < 0:
        return []
    if last - first == 0:
        return [points[first]]
    if last - first == 1:
        if points[first] == points[last]:
            return [points[first]]
        return [points[first], points[last]]
    if last - first == 2:
        if (points[first][0] == points[first + 1][0] and orient(points[first], points[first + 1], points[first + 2]) != 0) or orient(points[first], points[first + 1], points[first + 2]) < 0:
            return [points[first], points[first + 2], points[first + 1]]
        elif orient(points[first], points[first + 1], points[first + 2]) > 0:
            return [points[first], points[first + 1], points[first + 2]]
        else:
            return [points[first], points[first + 2]]

In [ ]:
def add_scene(lines2):
    global scenes, hulls
    lines = []
    points_scene = []
    for hull in hulls:
        n = len(hull)
        for i in range(n):
            lines.append([hull[i], hull[(i + 1) % n]])
            points_scene.append(hull[i])
    scenes.append(Scene([PointsCollection(data, color='skyblue'),
                 PointsCollection(points_scene, color='deeppink')],
            [LinesCollection(lines, color='deeppink'),
            LinesCollection(lines2, color='pink')]))

## 2. Main algorithm

In [ ]:
def recur(points, first, last):
    global hulls
    if last - first < 3:
        hull = trivial(points, first, last)
        hulls.append(hull)
        lines = []
        points_scene = []
        for hull in hulls:
            n = len(hull)
            for i in range(n):
                lines.append([hull[i], hull[(i + 1) % n]])
                points_scene.append(hull[i])
        scenes.append(Scene([PointsCollection(data, color='skyblue'),
                     PointsCollection(points_scene, color='deeppink')],
                [LinesCollection(lines, color='deeppink')]))
        return hull
    d = (last + first) // 2
    # division of points
    hull1 = recur(points, first, d)
    hull2 = recur(points, d + 1, last)
    if len(hull1) == 0:
        hulls.remove(hull1)
        return hull2
    if len(hull2) == 0:
        hulls.remove(hull2)
        return hull1

    hull = []
    n1 = len(hull1)
    n2 = len(hull2)
    max_x_ind = find_max_x(hull1)
    min_x_ind = find_min_x(hull2)
    down1, down2 = points_connect_down(max_x_ind, min_x_ind, hull1, hull2)
    up1, up2 = points_connect_up(max_x_ind, min_x_ind, hull1, hull2)
    add_scene([(hull1[down1], hull2[down2]), (hull1[up1], hull2[up2])])
    hulls.remove(hull1)
    hulls.remove(hull2)
    hull.append(hull1[down1])
    hull.append(hull2[down2])
    # joining two convex hulls with tangents
    if down2 != up2:
        a = down2 + 1
        while a % n2 != up2:
            hull.append(hull2[a % n2])
            a += 1
        hull.append(hull2[up2])
    if up1 != down1:
        hull.append(hull1[up1])
        b = up1 + 1
        while b % n1 != down1:
            hull.append(hull1[b % n1])
            b += 1
    hulls.append(hull)
    add_scene([])
    return hull

In [ ]:
def convex_hull(points):
    sorted_points = sorted(points, key = lambda k: (k[0], k[1]))
    n = len(sorted_points)
    return recur(sorted_points, 0, n - 1)

##### Loading a set of points from a json file:

In [ ]:
with open('points.json', 'r') as file:
    data = js.loads(file.read())


In [ ]:
scenes = []
hulls = []
scenes.append(Scene([PointsCollection(data, color='skyblue')]))
hull = convex_hull(data)
plot = Plot(scenes=scenes)
plot.draw()

## 3. Points generators:

In [ ]:
def on_retangle(n_sides, n_diagonals, vertexes):

    points = [vertexes[0], vertexes[1], vertexes[2], vertexes[3]]
    
    for i in range(n_sides):
        x1 = random.uniform(vertexes[0][0], vertexes[1][0])
        y1 = random.uniform(vertexes[0][1], vertexes[1][1])
        points.append((x1, y1))
        x2 = random.uniform(vertexes[0][0], vertexes[3][0])
        y2 = random.uniform(vertexes[0][1], vertexes[3][1])
        points.append((x2, y2))
        
    for i in range(n_diagonals):
        x1 = random.uniform(vertexes[0][0], vertexes[1][0])
        y1 = x1 + vertexes[0][1]
        points.append((x1, y1))
        x2 = random.uniform(vertexes[3][0], vertexes[2][0])
        y2 = -x2 + vertexes[3][1]
        points.append((x2, y2))
        
    return points

In [ ]:
def on_circle(n, s, r):
    points = []
    for i in range(n):
        a = random.uniform(0, 2*np.pi)
        x = np.cos(a) * (r ** 2) + s[0]
        y = np.sin(a) * (r ** 2) + s[1]
        points.append((x, y))
    return points

In [ ]:
def randoms(n, p_x, p_y):
    points = []
    for _ in range(n):
        x = random.uniform(p_x[0], p_x[1])
        y = random.uniform(p_y[0], p_y[1])
        points.append((x, y))
    return points

## 4. For saving points entered with the mouse:

In [ ]:
def save_plot(plot, name):
    points = []
    for i in range(len(plot.get_added_points())):
        for point in plot.get_added_points()[i].points:
            points.append(point)

    with open(f'{name}.json', 'w') as file:
       file.write(js.dumps(points))

In [ ]:
plot = Plot(scenes=[Scene()])
plot.draw()

## 5. Examples:

In [ ]:
data = randoms(100, [0, 100], [0, 100])
scenes = []
hull = []
points_hull = []
scenes.append(Scene([PointsCollection(data, color='skyblue')]))
hull = convex_hull(data)
plot = Plot(scenes=scenes)
plot.draw()

In [ ]:
data = on_circle(100, (0, 0), 1)
scenes = []
hulls = []
points_hull = []
scenes.append(Scene([PointsCollection(data, color='skyblue')]))
hull = convex_hull(data)
plot = Plot(scenes=scenes)
plot.draw()